In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

import os
print(os.getcwd())

/Users/omerorhan


In [2]:
def readExcelSheet1(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values

In [3]:
def readExcelRange(excelfile,sheetname="Sheet1",startrow=1,endrow=1,startcol=1,endcol=1):
    from pandas import read_excel
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]

In [4]:
def readExcel(excelfile,**args):
    if args:
        data=readExcelRange(excelfile,**args)
    else:
        data=readExcelSheet1(excelfile)
    if data.shape==(1,1):
        return data[0,0]
    elif (data.shape)[0]==1:
        return data[0]
    else:
        return data

In [5]:
def writeExcelData(x,excelfile,sheetname,startrow,startcol, reshape=None):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    if reshape:
        df=DataFrame(x.reshape(-1,len(x)))
    else:
        df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()

In [6]:
def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names

In [7]:
def Build1DHistogramClassifier(X,T,B,xmin,xmax):
    HF=np.zeros(B).astype('int32');
    HM=np.zeros(B).astype('int32');
    binindices=(np.round(((B-1)*(X-xmin)/(xmax-xmin)))).astype('int32');
    for i,b in enumerate(binindices):
        print(b)
        if T[i]=='Female':
            HF[b]+=1;
        else:
            HM[b]+=1;
    return [HF, HM]

In [8]:
def Apply1DHistogramClassifier(queries,HF,HM,xmin,xmax):
    B=np.alen(HF);
    binindices=np.clip((np.round(((B-1)*(queries-xmin)/(xmax-xmin)))).astype('int32'),0,B-1);
    countF=HF[binindices];
    countM=HM[binindices];
    resultlabel=np.full(np.alen(binindices),"Indeterminate",dtype=object);
    resultprob=np.full(np.alen(binindices),np.nan,dtype=object);
    indicesF=countF>countM;
    indicesM=countM>countF;
    resultlabel[indicesF]="Female";
    resultlabel[indicesM]="Male";
    probF=countF/(countF+countM);
    probM=countM/(countF+countM);
    resultprob[indicesF]=probF[indicesF];
    resultprob[indicesM]=probM[indicesM];
    return resultlabel, resultprob

In [9]:
excelfile=r"/Users/omerorhan/Desktop/UCSC/MachineLearning/Assignment_1_Data_and_Template.xlsx";
sheets=getSheetNames(excelfile);sheets

['Data',
 'Classifiers - Full Data',
 'Classifiers - Partial Data',
 'Queries',
 'Observations',
 'ID']

In [ ]:
data=readExcel(excelfile)
X=np.array(data[:,0]*12+data[:,1],dtype=float)
T=np.array([str(g) for g in data[:,2]])
print(X)
print(T)

In [ ]:
queries=(readExcel(excelfile,
                  sheetname='Classifiers - Full Data',
                  startrow=17,
                  endrow=17,
                  startcol=2,
                  endcol=7)).astype(float);queries

In [ ]:
#Histogram Classifier

B=32;
xmin=np.amin(X);
xmax=np.amax(X);
[HF,HM]=Build1DHistogramClassifier(X,T,B,xmin,xmax);


In [ ]:
plt.figure(figsize=(10,5));
opacity = 0.5
[bincenters,binwidth]=np.linspace(xmin, xmax, num=B, retstep=True);
rects1 = plt.bar(bincenters-(binwidth/2), HF, binwidth,
                 alpha=opacity,
                 color='pink',
                 edgecolor='black',
                 label='Female')
rects2 = plt.bar(bincenters+(binwidth/2), HM, binwidth,
                 alpha=opacity,
                 color='b',
                 edgecolor='black',
                 label='Male')
plt.xlabel('Height')
plt.ylabel('Count')
plt.xticks(bincenters, bincenters.astype('int32'), fontsize=10)
plt.legend()
plt.show()

In [ ]:
[resultHlabel, resultHprob]=Apply1DHistogramClassifier(queries,HF,HM,xmin,xmax)


In [ ]:
[resultHlabel, resultHprob]=Apply1DHistogramClassifier(queries,HF,HM,xmin,xmax)


In [10]:
#full data
excelfile=r"/Users/omerorhan/Desktop/UCSC/MachineLearning/Assignment_1_Data_and_Template.xlsx";
sheets=getSheetNames(excelfile);sheets

data=readExcel(excelfile)
X=np.array(data[:,0]*12+data[:,1],dtype=float)
T=np.array([str(g) for g in data[:,2]])
print(X)
print(T)

B=32;
xmin=np.amin(X);
xmax=np.amax(X);
writeExcelData(np.array([xmin]),excelfile,'Classifiers - Full Data',1,2)
writeExcelData(np.array([xmax]),excelfile,'Classifiers - Full Data',2,2)

#print("xmax= %.2f",xmax)
#print("xmin= %.2f",xmin)

[HF,HM]=Build1DHistogramClassifier(X,T,B,xmin,xmax);
#print(Build1DHistogramClassifier(X,T,B,xmin,xmax))
#def writeExcelData(x,excelfile,sheetname,startrow,startcol):
#print(HF)
writeExcelData(HF,excelfile,'Classifiers - Full Data',5,3, True)
writeExcelData(HM,excelfile,'Classifiers - Full Data',6,3, True)

[resultHlabel, resultHprob]=Apply1DHistogramClassifier([55,60,65,70,75,80],HF,HM,xmin,xmax)
writeExcelData(resultHlabel,excelfile,'Queries',3,2)
writeExcelData(resultHprob,excelfile,'Queries',3,3)


#print([resultHlabel, resultHprob])

print("finished")


[70. 76. 70. ... 69. 60. 64.]
['Male' 'Male' 'Male' ... 'Male' 'Female' 'Female']
18
24
18
17
15
20
19
17
13
10
15
19
18
23
22
6
17
12
22
24
17
21
14
22
10
10
13
19
12
22
12
17
20
11
11
3
10
13
28
12
14
9
19
14
16
14
17
14
4
7
13
5
17
19
13
12
19
6
13
19
16
14
15
11
15
15
12
20
16
14
12
14
15
15
22
5
18
5
14
9
16
21
25
11
14
24
17
15
25
17
10
17
13
14
14
13
18
19
16
19
18
20
12
12
13
5
9
23
11
9
20
19
15
16
12
8
22
10
9
15
14
16
14
10
21
10
16
15
11
13
17
11
20
17
26
13
23
13
21
15
17
16
15
14
19
9
17
19
18
16
22
18
17
13
16
16
18
7
11
18
12
24
23
13
8
22
23
28
19
5
19
14
10
13
18
11
19
18
16
11
18
22
7
15
15
19
20
17
12
15
7
18
5
18
14
19
14
15
14
14
16
18
12
11
13
17
15
10
9
17
15
5
20
14
21
4
11
18
11
15
24
15
14
11
11
9
19
11
24
12
9
10
17
24
15
20
14
10
17
18
12
20
15
18
10
17
22
20
13
21
17
14
21
24
17
19
18
16
14
17
22
18
13
15
9
17
8
14
19
9
12
12
17
10
15
11
19
13
14
20
11
16
16
16
15
20
23
19
23
23
12
15
19
26
13
18
20
15
4
13
18
11
16
10
19
21
16
9
15
18
15
16
12
17
17
14
13

12
16
14
19
13
11
8
20
18
22
11
15
9
12
16
12
18
18
18
23
19
10
13
14
8
12
19
15
21
16
10
17
12
17
18
17
15
16
10
18
19
12
8
20
11
12
11
10
22
16
16
16
15
15
16
21
21
18
19
22
14
22
12
20
22
24
19
13
13
17
14
16
19
10
11
11
24
20
18
12
18
16
20
16
16
14
14
13
15
17
22
15
17
23
15
22
19
19
10
15
17
13
16
14
11
12
14
12
12
19
16
23
19
16
9
17
16
14
22
17
16
14
17
16
12
13
12
10
18
12
10
21
20
17
16
16
8
15
15
6
21
14
18
17
15
14
14
12
6
24
16
25
11
17
15
13
18
17
12
21
13
17
15
18
16
8
16
17
11
14
14
14
24
19
18
22
15
22
14
9
20
13
16
15
12
9
8
14
14
12
13
18
16
12
7
16
19
19
7
15
16
20
11
17
10
9
8
15
14
20
13
10
22
5
20
13
19
23
11
22
8
17
17
16
11
7
19
21
14
15
7
12
6
13
19
20
11
18
15
23
18
14
19
18
20
11
10
7
12
12
18
17
9
16
15
14
17
14
22
19
24
20
17
21
14
13
14
22
13
12
14
10
18
13
20
9
20
16
19
17
22
19
13
20
21
19
17
14
9
17
19
11
24
18
13
14
22
8
17
11
13
16
15
10
19
14
18
22
16
18
9
9
12
13
18
14
20
11
22
14
21
20
20
14
13
16
14
21
23
20
14
19
13
10
8
18
14
12
20
11
12
22
14


11
15
10
20
18
25
16
19
10
11
17
13
16
8
15
15
14
20
20
26
12
11
19
17
17
15
15
21
9
11
14
10
12
19
10
18
15
23
22
19
12
14
7
10
24
6
15
14
13
13
10
16
23
15
10
20
22
14
11
18
18
11
20
18
18
18
13
10
20
19
14
16
16
14
15
28
19
15
17
15
18
16
7
16
11
20
9
15
10
13
18
13
16
17
14
25
14
18
15
20
18
9
7
21
14
13
23
10
13
16
21
13
8
9
9
22
13
9
22
13
12
15
17
23
12
11
9
19
15
15
22
10
14
15
20
19
13
8
11
17
13
17
6
16
9
15
15
13
18
17
11
16
12
13
20
19
7
13
13
17
16
15
18
19
13
18
14
8
8
23
13
13
19
16
15
14
10
12
16
18
28
19
8
14
8
22
21
17
15
22
18
15
13
15
11
12
18
10
26
23
22
11
10
19
11
15
7
13
12
16
24
19
20
10
20
22
17
18
19
18
26
17
20
16
15
5
13
21
15
20
16
16
6
20
22
14
10
11
13
25
18
17
12
9
19
27
12
16
13
28
16
14
10
9
22
13
20
13
11
19
19
19
21
17
11
14
21
22
13
13
22
23
11
10
13
19
19
15
9
18
15
10
17
24
14
14
17
7
11
13
14
20
20
14
15
15
20
13
19
21
15
16
19
16
18
18
27
14
15
20
20
17
8
13
9
23
21
8
13
22
13
13
7
21
23
17
17
20
17
16
14
17
14
21
16
15
19
23
15
13
13
20
13
7
1

22
21
21
17
16
22
12
14
19
12
14
14
17
13
10
20
9
18
11
18
13
17
16
14
6
21
21
15
16
19
7
18
14
15
18
19
11
20
12
21
21
9
17
16
18
13
17
17
11
18
17
17
16
14
14
19
22
17
15
20
11
16
17
16
15
18
20
13
11
12
13
13
20
13
22
14
12
14
12
20
12
11
14
22
16
12
21
15
20
12
20
14
17
9
7
19
18
13
12
12
15
17
12
14
13
17
22
26
7
19
11
17
12
13
7
20
10
15
10
10
20
17
20
8
22
11
11
23
17
16
18
10
21
12
17
13
23
16
18
15
14
15
4
15
13
6
11
18
20
18
21
17
18
22
20
14
14
21
23
21
22
17
16
18
22
11
19
18
15
14
14
18
22
17
9
13
11
19
17
18
10
12
11
19
19
16
10
20
18
18
14
19
9
6
16
15
20
22
20
20
20
15
17
16
17
16
17
22
15
12
20
16
20
13
18
17
17
17
12
12
18
17
26
22
11
23
15
14
14
20
6
19
19
11
17
14
15
21
18
22
19
17
19
23
21
18
16
19
12
15
16
15
17
12
14
14
19
17
18
20
16
7
9
16
20
10
21
24
12
8
19
20
11
15
10
9
14
21
8
17
13
14
23
19
20
10
14
23
17
24
15
17
7
20
14
15
16
7
14
21
21
16
15
16
21
19
17
13
16
12
22
27
15
14
19
18
18
15
15
15
13
10
12
15
13
19
16
18
23
9
20
11
13
24
16
17
17
17
11
16
16


18
17
12
18
14
17
11
16
10
13
19
10
17
13
7
22
14
24
19
10
13
14
3
22
16
8
21
13
14
11
14
15
16
19
13
11
13
20
19
13
18
13
16
9
17
16
17
17
20
22
7
21
13
21
11
18
22
10
16
10
24
22
10
7
16
15
20
21
15
17
15
19
16
17
8
15
11
19
19
22
15
15
9
14
19
19
14
22
15
13
20
21
16
22
18
15
18
18
9
13
19
25
18
23
15
11
9
17
19
14
18
17
17
18
12
14
22
24
17
16
13
7
14
12
15
16
17
18
14
15
12
12
13
15
12
9
21
15
17
20
11
11
19
19
15
13
12
10
21
18
12
13
18
29
23
10
16
21
14
14
22
18
16
11
12
12
13
17
12
23
21
19
17
14
7
22
22
16
19
15
17
8
11
12
22
9
10
10
19
20
20
21
16
9
13
12
22
20
13
16
16
16
14
16
12
13
22
15
21
25
15
14
11
8
14
19
10
7
18
21
20
22
13
20
17
11
18
18
16
19
16
14
16
4
16
7
21
15
8
18
21
11
15
17
19
23
17
21
18
9
8
16
15
14
20
16
15
10
19
20
21
9
17
9
20
20
12
12
13
21
17
13
20
14
18
7
15
14
17
22
13
16
9
18
15
21
14
14
17
18
20
18
19
12
16
21
15
13
11
8
22
25
20
21
19
9
17
15
25
18
11
16
9
14
11
25
14
25
15
17
17
15
24
13
10
11
11
16
17
20
15
13
20
18
7
21
18
10
14
17
12
9
11
23


18
20
10
17
20
14
16
8
12
11
15
18
22
19
13
11
19
7
14
16
16
15
15
16
21
16
22
19
14
21
20
15
14
14
19
21
15
9
15
18
13
9
12
16
19
16
21
16
11
19
18
21
12
13
22
24
14
15
8
21
9
12
16
17
18
12
14
13
14
15
20
22
19
18
28
14
20
15
20
13
12
16
15
11
10
22
14
19
15
21
9
15
21
11
18
24
17
22
11
20
21
14
19
20
17
18
14
17
28
17
12
15
21
14
25
16
26
18
22
19
19
13
20
13
10
17
17
15
11
8
15
15
16
13
16
16
18
14
23
14
12
10
25
17
13
12
8
6
16
13
19
7
19
22
21
19
8
18
17
11
11
17
22
13
18
19
12
2
11
12
14
17
5
9
15
16
8
22
12
16
13
15
11
14
22
19
22
20
13
13
19
19
16
18
21
17
19
9
19
8
21
12
20
15
13
15
19
11
18
19
11
16
13
22
22
16
16
17
20
16
17
17
10
20
15
22
9
20
19
25
19
13
9
18
19
14
11
16
16
18
16
11
18
17
10
17
17
11
20
22
18
10
23
8
13
11
15
22
22
19
17
22
20
16
10
18
25
20
12
20
11
14
13
17
8
12
finished


In [11]:
#FULL DATA
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

print(mean([1,2,3]))

X=np.array(data[:,0]*12+data[:,1],dtype=float)
data[:,1]= X
#print(data)
T=np.array([str(g) for g in data[:,2]])

#np.select(data[:,2]=='Female', data[,1])
femaledata = (data[data[:,2]=='Female'])
maledata = (data[data[:,2]=='Male'])
print(X)
#print(maledata[:,[1]].std()) 
meanFemale= femaledata[:,[1]].mean()
meanMale = maledata[:,[1]].mean()
writeExcelData([femaledata[:,[1]].mean()],excelfile,'Classifiers - Full Data',8,3)
writeExcelData([maledata[:,[1]].mean()],excelfile,'Classifiers - Full Data',9,3)

stdfemale= femaledata[:,[1]].std()
stdmale=maledata[:,[1]].std()
writeExcelData([femaledata[:,[1]].std()],excelfile,'Classifiers - Full Data',11,3)
writeExcelData([maledata[:,[1]].std()],excelfile,'Classifiers - Full Data',12,3)

writeExcelData([len(femaledata[:,[1]])],excelfile,'Classifiers - Full Data',14,3)
writeExcelData([len(maledata[:,[1]])],excelfile,'Classifiers - Full Data',15,3)

print('finished')
print(len(femaledata[:,[1]]))

2.0
[70. 76. 70. ... 69. 60. 64.]
finished
8900


In [ ]:
def Apply1DHistogramClassifier(queries,HF,HM,xmin,xmax):
    B=np.alen(HF);
    binindices=np.clip((np.round(((B-1)*(queries-xmin)/(xmax-xmin)))).astype('int32'),0,B-1);
    countF=HF[binindices];
    countM=HM[binindices];
    resultlabel=np.full(np.alen(binindices),"Indeterminate",dtype=object);
    resultprob=np.full(np.alen(binindices),np.nan,dtype=object);
    indicesF=countF>countM;
    indicesM=countM>countF;
    resultlabel[indicesF]="Female";
    resultlabel[indicesM]="Male";
    probF=countF/(countF+countM);
    print(probF)
    probM=countM/(countF+countM);
    resultprob[indicesF]=probF[indicesF];
    resultprob[indicesM]=probM[indicesM];
    return resultlabel, resultprob



In [ ]:
import math
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
resultlabel1=[]
problabel=[]
liste = [65]

print(calculateProbability(65,meanMale,stdmale))

In [12]:
import math
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

#meanFemale,meanMale,stdfemale,stdmale
resultlabel1=[]
problabel=[]
liste = [55,60,65,70,75,80]
print(len(maledata))
print(len(femaledata))

for item in liste: 
    maleProb= len(maledata)*calculateProbability(item,meanMale,stdmale)
    femaleprob= len(femaledata)*calculateProbability(item,meanFemale,stdfemale)
    maleProb= maleProb/(maleProb+femaleprob)
    femaleprob= femaleprob/(maleProb+femaleprob)

    if maleProb >femaleprob:
        resultlabel1.append('Male')
        problabel.append(maleProb)
    else:
        resultlabel1.append('Female')
        problabel.append(femaleprob)

writeExcelData(resultlabel1,excelfile,'Queries',3,4)
writeExcelData(problabel,excelfile,'Queries',3,5)
print('finished')

7800
8900
finished


In [ ]:
import math
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

#meanFemale,meanMale,stdfemale,stdmale
resultlabel1=[]
problabel=[]
liste = [55]
for item in liste: 
    maleProb= len(maledata[:,[1]])*calculateProbability(item,meanMale,stdmale)   
    femaleprob= len(femaledata[:,[1]]) * calculateProbability(item,meanFemale,stdfemale)

    print(maleProb)
    print(femaleprob)
print('finished')

In [ ]:
def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.iteritems():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities

In [ ]:
#partial data
excelfile=r"/Users/omerorhan/Desktop/UCSC/MachineLearning/Assignment_1_Data_and_Template.xlsx";
sheets=getSheetNames(excelfile);sheets

data=readExcel(excelfile)
data = data[0:50]
X=np.array(data[:,0]*12+data[:,1],dtype=float)
T=np.array([str(g) for g in data[:,2]])
B=32;

writeExcelData(np.array([xmin]),excelfile,'Classifiers - Partial Data',1,2)
writeExcelData(np.array([xmax]),excelfile,'Classifiers - Partial Data',2,2)



[HF,HM]=Build1DHistogramClassifier(X,T,B,xmin,xmax);
#print(Build1DHistogramClassifier(X,T,B,xmin,xmax))
#def writeExcelData(x,excelfile,sheetname,startrow,startcol):
writeExcelData(HF,excelfile,'Classifiers - Partial Data',5,3, True)
writeExcelData(HM,excelfile,'Classifiers - Partial Data',6,3, True)


[resultHlabel, resultHprob]=Apply1DHistogramClassifier([55,60,65,70,75,80],HF,HM,xmin,xmax)
writeExcelData(resultHlabel,excelfile,'Queries',12,2)
writeExcelData(resultHprob,excelfile,'Queries',12,3)
print([resultHlabel, resultHprob])

print('finished')

In [ ]:

plt.figure(figsize=(10,5));
opacity = 0.5
[bincenters,binwidth]=np.linspace(xmin, xmax, num=B, retstep=True);
rects1 = plt.bar(bincenters-(binwidth/2), HF, binwidth,
                 alpha=opacity,
                 color='pink',
                 edgecolor='black',
                 label='Female')
rects2 = plt.bar(bincenters+(binwidth/2), HM, binwidth,
                 alpha=opacity,
                 color='b',
                 edgecolor='black',
                 label='Male')
plt.xlabel('Height')
plt.ylabel('Count')
plt.xticks(bincenters, bincenters.astype('int32'), fontsize=10)
plt.legend()
plt.show()


In [ ]:
def Apply1DHistogramClassifier(queries,HF,HM,xmin,xmax):
    B=np.alen(HF);
    print(B)
    binindices=np.clip((np.round(((B-1)*(queries-xmin)/(xmax-xmin)))).astype('int32'),0,B-1);
    print("sdasd"+str(binindices))
    countF=HF[binindices];
    countM=HM[binindices];

    resultlabel=np.full(np.alen(binindices),"Indeterminate",dtype=object);
    resultprob=np.full(np.alen(binindices),np.nan,dtype=object);
    print(resultlabel)

    indicesF=countF>countM;
    indicesM=countM>countF;
    resultlabel[indicesF]="Female";
    resultlabel[indicesM]="Male";
    probF=countF/(countF+countM);
    print(probF)
    probM=countM/(countF+countM);
    resultprob[indicesF]=probF[indicesF];
    resultprob[indicesM]=probM[indicesM];
    return resultlabel, resultprob
print(HM)
[resultHlabel, resultHprob]=Apply1DHistogramClassifier([55,60,65,70,75,80],HF,HM,xmin,xmax)


In [ ]:
binindices=np.clip(8,0,31);
print(binindices)

In [ ]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

X=np.array(data[:,0]*12+data[:,1],dtype=float)
data[:,1]= X
#print(data)
T=np.array([str(g) for g in data[:,2]])

#np.select(data[:,2]=='Female', data[,1])
femaledata = (data[data[:,2]=='Female'])
maledata = (data[data[:,2]=='Male'])
print(X)
#print(maledata[:,[1]].std()) 
meanFemale= femaledata[:,[1]].mean()
meanMale = maledata[:,[1]].mean()
writeExcelData([femaledata[:,[1]].mean()],excelfile,'Classifiers - Partial Data',8,3)
writeExcelData([maledata[:,[1]].mean()],excelfile,'Classifiers - Partial Data',9,3)

stdfemale= femaledata[:,[1]].std()
stdmale=maledata[:,[1]].std()
writeExcelData([femaledata[:,[1]].std()],excelfile,'Classifiers - Partial Data',11,3)
writeExcelData([maledata[:,[1]].std()],excelfile,'Classifiers - Partial Data',12,3)

writeExcelData([len(femaledata[:,[1]])],excelfile,'Classifiers - Partial Data',14,3)
writeExcelData([len(maledata[:,[1]])],excelfile,'Classifiers - Partial Data',15,3)



print('finished')

In [ ]:
import math
def calculateProbability(x, mean, stdev):
    print(math.pow(x-mean,2))
    print((2*math.pow(stdev,2)))
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

#meanFemale,meanMale,stdfemale,stdmale
resultlabel1=[]
problabel=[]
liste = [55,60,65,70,75,80]
print(len(maledata))
print(meanFemale)
print(stdmale)
print(stdfemale)
for item in liste:
    maleProb= len(maledata)*calculateProbability(item,meanMale,stdmale)
    femaleprob= len(femaledata)*calculateProbability(item,meanFemale,stdfemale)
    maleProb= maleProb/(maleProb+femaleprob)
    femaleprob= femaleprob/(maleProb+femaleprob)

    if len(maledata[:,[1]])*maleProb >len(femaledata[:,[1]])*femaleprob:
        resultlabel1.append('Male')
        problabel.append(maleProb)
    else:
        resultlabel1.append('Female')
        problabel.append(femaleprob)


#print(resultlabel1)
#print(problabel)

writeExcelData(resultlabel1,excelfile,'Queries',12,4)
writeExcelData(problabel,excelfile,'Queries',12,5)
print('finished')